In [1]:
import pandas as pd

In [2]:
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [4]:

data=pd.read_csv(r'C:\Users\deyar\OneDrive\Desktop\Flight_Ticket_Price_Prediction\notebooks\data\flight.csv')

In [5]:
cat=[]
num=[]
for i in data.columns:
    if data[i].dtype=='object':
        cat.append(i)
    else:
        num.append(i)

In [6]:
data_cat=data[cat]

In [7]:
data_num=data[num]

In [8]:
for i in data_cat.columns:
    x=pd.unique(data_cat[i])
    print(i,"=",x)

airline = ['SpiceJet' 'AirAsia' 'Vistara' 'GO_FIRST' 'Indigo' 'Air_India']
flight = ['SG-8709' 'SG-8157' 'I5-764' ... '6E-7127' '6E-7259' 'AI-433']
source_city = ['Delhi' 'Mumbai' 'Bangalore' 'Kolkata' 'Hyderabad' 'Chennai']
departure_time = ['Evening' 'Early_Morning' 'Morning' 'Afternoon' 'Night' 'Late_Night']
stops = ['zero' 'one' 'two_or_more']
arrival_time = ['Night' 'Morning' 'Early_Morning' 'Afternoon' 'Evening' 'Late_Night']
destination_city = ['Mumbai' 'Bangalore' 'Kolkata' 'Hyderabad' 'Chennai' 'Delhi']
class = ['Economy' 'Business']


In [9]:
for  j in data_cat.columns:
    data_cat[j]=data_cat[j].astype('category').cat.codes

In [10]:
data=pd.concat([data_cat,data_num],axis=1)

In [11]:
y=data.iloc[:,-1:]

In [12]:
x=data.iloc[:,:-1]

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
from sklearn.model_selection import train_test_split,cross_val_score

In [15]:
from sklearn.linear_model import ElasticNet

In [16]:
import optuna

In [17]:
s=StandardScaler()

In [18]:
x_scaled=s.fit_transform(x)

In [19]:
x_scaled=pd.DataFrame(x_scaled,columns=x.columns)

In [20]:
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=.2,random_state=12)

In [21]:
from sklearn.metrics import r2_score

In [22]:
def objective(trial):
    alpha=trial.suggest_float('alpha',.1,10,log=True)
    l1_ratio=trial.suggest_float('l1_ratio',.1,1,log=True)
    max_iter=trial.suggest_int('max_iter',1000,10000)
    model=ElasticNet(alpha=alpha,
    l1_ratio=l1_ratio,
    max_iter=max_iter
    )
    model.fit(x_train,y_train)
    p=model.predict(x_test)
    v=r2_score(y_test,p)
    return v

In [23]:
study=optuna.create_study(direction='maximize')

[I 2025-03-26 13:44:07,786] A new study created in memory with name: no-name-36b8fb2a-3c8e-417c-b27a-32e5a679b10c


In [24]:
study.optimize(objective,n_trials=100)

[I 2025-03-26 13:44:08,468] Trial 0 finished with value: 0.8358950711294282 and parameters: {'alpha': 0.5283389147612767, 'l1_ratio': 0.14293321941493015, 'max_iter': 1497}. Best is trial 0 with value: 0.8358950711294282.
[I 2025-03-26 13:44:08,539] Trial 1 finished with value: 0.8433033730113133 and parameters: {'alpha': 0.9468390826351726, 'l1_ratio': 0.5709556931374198, 'max_iter': 6184}. Best is trial 1 with value: 0.8433033730113133.
[I 2025-03-26 13:44:08,637] Trial 2 finished with value: 0.8879932242634448 and parameters: {'alpha': 0.2572728784052659, 'l1_ratio': 0.4780836472941477, 'max_iter': 3465}. Best is trial 2 with value: 0.8879932242634448.
[I 2025-03-26 13:44:08,709] Trial 3 finished with value: 0.8537385245554243 and parameters: {'alpha': 0.40656709464824253, 'l1_ratio': 0.1602166118531678, 'max_iter': 5989}. Best is trial 2 with value: 0.8879932242634448.
[I 2025-03-26 13:44:08,809] Trial 4 finished with value: 0.8856194559774613 and parameters: {'alpha': 0.1782684712

In [25]:
study.best_params

{'alpha': 0.9977858104160371, 'l1_ratio': 0.9994072380775121, 'max_iter': 3844}

In [26]:
study.best_value

0.904646192550211

In [27]:
from sklearn.ensemble import RandomForestRegressor

In [28]:
r=RandomForestRegressor()

In [29]:
r.fit(x_train,y_train)

RandomForestRegressor()

In [30]:
p=r.predict(x_test)

In [31]:
r2_score(y_test,p)

0.991368296021624